In [1]:
''' initialise development environment '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2


In [2]:
''' import optics package '''

# add custom python packages directory to path
import sys
sys.path.append('/home/brendan/dev/optics')

%matplotlib widget

# import path tracing and image transformation engine
import optics


In [3]:
''' Imports '''

# nd array manipulation
import numpy as np

# image manipulation
from scipy import ndimage

# plotting with matplotlib, interactive notebook
import matplotlib.pyplot as plt


In [4]:
''' generate image, calculate distance map relative zero (centre) '''

# set edge length; ensure odd
edge_len = 2001


# generate target range
div = [ (2.0 / (edge_len - 1)) for _ in range(2) ]
rng = [ [-1., 1.] for _ in range(2) ]

# prepare blank grid
grid = [ np.arange(rng[i][0], rng[i][1] + div[i], div[i]) for i in range(2) ]
zero = np.array([ [ 0. for xi in grid[0] ] for yi in grid[1] ])


# set target origin
orig = np.array([0., 0.])

# calculate distance map
dist_r = np.array([ [ np.sqrt(sum((np.array([xi, yi])-orig)**2)) for xi in grid[0] ] for yi in grid[1] ])

dist_h = np.array([ [ np.sqrt(sum((np.array([xi,])-orig)**2)) for xi in grid[0] ] for yi in grid[1] ])
dist_v = np.array([ [ np.sqrt(sum((np.array([yi,])-orig)**2)) for xi in grid[0] ] for yi in grid[1] ])

dist_hn = np.array([ [ xi-orig[0] for xi in grid[0] ] for yi in grid[1] ])
dist_vn = np.array([ [ yi-orig[1] for xi in grid[0] ] for yi in grid[1] ])[::-1]


In [6]:
''' calculate per-pixel angle relative zero (centre)'''

# initialise angle matrix
angl = zero.copy()

# iterate over pixel
for i in range(dist_r.shape[0]):
    for j in range(dist_r.shape[1]):
        
        # calculate angle from distance, store in matrix
        angl[i,j] = np.arctan( (dist_vn[i,j]/dist_hn[i,j]) )
        angl[i,j] = (lambda x: x if x >= 0 else x + np.pi)(angl[i,j])

# blank central region
j = np.where(dist_vn < 0)
angl[j] += np.pi

#angl/=angl.max()

# generate spokes target
spks = 160
test_img = np.sin(angl*spks)

# tophat floor/ciel filter for pattern
if True:
    test_img[np.where(test_img >= .0)] = 1.
    test_img[np.where(test_img < .0)] = 0.
    
# blank centre of spoke
if True:
    test_img[np.where(dist_r <= .10)] = .0


In [15]:
''' output raw spoke images '''

#spks = 160
# iterate spoke counts
#for spks in [80, 120, 160]:
for spks in range(60, 260, 20):

    # generate spokes target
    test_img = np.sin(angl*spks)

    # tophat floor/ciel filter for pattern
    test_img[np.where(test_img >= .0)] = 1.
    test_img[np.where(test_img < .0)] = 0.

    # blank centre of spoke
    test_img[np.where(dist_r <= .1)] = .0
    
    # init figure and format
    _w = 12; _h = 12; fig = plt.figure(figsize = (_w, _h))
    #fig.canvas.layout.width = '{}in'.format(_w); fig.canvas.layout.height= '{}in'.format(_h)
    ax = fig.add_subplot(111); ax.grid([]); ax.set_xticks([]); ax.set_yticks([])

    _img = test_img.copy()

    # display image
    ax.imshow(_img, cmap = 'bone_r')#, vmin = -1, vmax = 1)

    plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)
    #plt.tight_layout()
    plt.savefig('../data/spoke-raw/spoke_{}.png'.format(spks), dpi = 200)

    plt.close()
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
''' functions for ellipse '''

#def ellipse(theta, a, b):   
#    r = a*b / (np.sqrt((b*np.cos(theta))**2 + (a*np.sin(theta))**2))    
#    return r

# for given angle, eccentricity, and long-radius, calculate and return ellipse distance (radius)
def ellipse_e(theta, e, b):
    
    # parametric equation ellipse (polar coordinates)
    r = b / (1 - (e*np.cos(theta))**2)
    return r


In [11]:
''' generate test spoke image with ellipse overlay '''

_w = 12; _h = 12; fig = plt.figure(figsize = (_w, _h))
fig.canvas.layout.width = '{}in'.format(_w); fig.canvas.layout.height= '{}in'.format(_h)
ax = fig.add_subplot(111); ax.grid([]); ax.set_xticks([]); ax.set_yticks([])

_img = test_img.copy()

## add circle
#rl = .8
rt = .005
#_img[np.where((dist_r >= rl) & (dist_r <= rl + rt))] = .6

# add ellipse, long-radius b, eccentricity e
b = .25
e = .5

theta = np.pi/8

r = ellipse_e(angl+theta, e, b)
j = np.where( (dist_r <= r + rt) & (dist_r >= r - rt) )
_img[j] = .6


#_img = ndimage.gaussian_filter(_img, sigma = 1.)

# display image
ax.imshow(_img, cmap = 'bone_r')#, vmin = -1, vmax = 1)

plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)
#plt.tight_layout()
plt.savefig('../data/spoke-ellipse/spoke_{}-id_{}.png'.format(spks, '0'), dpi = 200)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
''' generate batch spoke images with ellipse over parameter ranges '''

# initialise image counter, storage list
i = 1
img_map = []

rt = .005 # line thickness

# iterate over each: ellipse radius and eccentricity, rotational angle (0 to 90 deg, every 10 deg)
for rl in np.arange(.15, .95, .025):
    for e in np.arange(.0, .5, .05):
        
        for theta in np.arange(0., np.pi/2, np.pi/18):

            # initialise figure
            _w = 12; _h = 12; fig = plt.figure(figsize = (_w, _h))
            fig.canvas.layout.width = '{}in'.format(_w); fig.canvas.layout.height= '{}in'.format(_h)
            ax = fig.add_subplot(111); ax.grid([]); ax.set_xticks([]); ax.set_yticks([])

            # get raw spoke image copy
            _img = test_img.copy()

            ## add ellipse
            r = ellipse_e(angl + theta, e, rl)
            j = np.where( (dist_r <= r + rt) & (dist_r >= r - rt) )
            _img[j] = .6

            # display image
            ax.imshow(_img, cmap = 'bone_r')#, vmin = -1, vmax = 1)

            # remove image border
            plt.subplots_adjust(left = .0, right = 1., top = 1., bottom = .0)
            #plt.tight_layout()

            # save image to file
            #plt.savefig('../data/spoke/spoke-{}-{:.2f}.png'.format(spks, rl), dpi = 200)
            plt.savefig('../data/spoke-ellipse/spoke_{}-id_{}.png'.format(spks,i), dpi = 200)


            # store image in list, increment counter
            img_map.append([i, rl, e, theta])
            i += 1

            # close figuew
            plt.close()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
''' save image map file '''

# stack list parameters into array
img_map = np.stack(img_map)

# define headers
head = ['Image Index', 'Long-Radius', 'Eccentricity', 'Rotation Angle']

# build headers csv string from provided headers list
header = ','.join( [str(h) for h in head] )

# ser file path
_file_path = f'../data/spoke-ellipse/spoke_{spks}_image-map.csv'

# export data array to file with provided headers
np.savetxt(X = img_map, delimiter = ',', fname = _file_path, header = header, fmt = '%s', comments = '')
